In [1]:
markdown_text = """
# Vector

## Vector trong R^n 

Vector là một bộ n số thực, được biểu diễn trong không gian R^n. Ví dụ: (1, 2, 3) ∈ R^3.

## Phép cộng

Phép cộng hai vector cùng kích thước được thực hiện bằng cách cộng từng thành phần tương ứng.
"""


In [5]:
import re
import os
import torch
import glob

from langchain.text_splitter import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
raw_data_folder = "Raw_data"
model_name = "intfloat/multilingual-e5-base"
persist_directory = "chroma_db"

In [21]:
txt_files = glob.glob(os.path.join(raw_data_folder, "*.txt"))

def read_txt_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    
def clean_text(text):
    #text.to_lower()  # Chuyển đổi chữ hoa thành chữ thường
    #text = re.sub(r'#* ', '', text)  # Remove Markdown headers
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = text.strip() 
    return text

markdown_text = ""

for file_path in txt_files:
    try:
        content = read_txt_file(file_path)
        #content = clean_text(content)
        if content:
            markdown_text += f"\n\n## {os.path.basename(file_path)}\n\n{content}"
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

In [22]:
headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
            ("####", "Header 4"),
            ]

splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on = headers_to_split_on
)

chunks = splitter.split_text(content)


In [23]:
documents = []
for chunk in chunks:
    documents.append({
        "text": chunk.page_content,
        "metadata": chunk.metadata  # Giữ lại thông tin header
    })

In [24]:
documents

[{'text': 'Ma trận là một bảng số được sắp xếp theo hàng và cột. Ma trận A có m hàng và n cột được ký hiệu là A(m×n) hoặc A[m,n].  \nMa trận tổng quát có dạng:\nA = [a11  a12  ...  a1n]\n[a21  a22  ...  a2n]\n[... ...  ...  ...]\n[am1  am2  ...  amn]  \nTrong đó aij là phần tử ở hàng i, cột j.',
  'metadata': {'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính',
   'Header 2': '1. Định nghĩa Ma trận'}},
 {'text': 'Ma trận có số hàng bằng số cột (m = n).',
  'metadata': {'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính',
   'Header 2': '2. Các loại Ma trận đặc biệt',
   'Header 3': 'Ma trận vuông'}},
 {'text': 'Ma trận vuông I có tất cả các phần tử trên đường chéo chính bằng 1, các phần tử khác bằng 0.  \nI = [1  0  0]\n[0  1  0]\n[0  0  1]',
  'metadata': {'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính',
   'Header 2': '2. Các loại Ma trận đặc biệt',
   'Header 3': 'Ma trận đơn vị'}},
 {'text': 'Ma trận có tất cả các phần tử bằng 0.',
  'metadata': {'Header 1': 'Kiến thức Ma trận

In [25]:
langchain_docs = [
    Document(page_content=doc["text"], metadata=doc["metadata"])
    for doc in documents
]

In [26]:
langchain_docs

[Document(metadata={'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính', 'Header 2': '1. Định nghĩa Ma trận'}, page_content='Ma trận là một bảng số được sắp xếp theo hàng và cột. Ma trận A có m hàng và n cột được ký hiệu là A(m×n) hoặc A[m,n].  \nMa trận tổng quát có dạng:\nA = [a11  a12  ...  a1n]\n[a21  a22  ...  a2n]\n[... ...  ...  ...]\n[am1  am2  ...  amn]  \nTrong đó aij là phần tử ở hàng i, cột j.'),
 Document(metadata={'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính', 'Header 2': '2. Các loại Ma trận đặc biệt', 'Header 3': 'Ma trận vuông'}, page_content='Ma trận có số hàng bằng số cột (m = n).'),
 Document(metadata={'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính', 'Header 2': '2. Các loại Ma trận đặc biệt', 'Header 3': 'Ma trận đơn vị'}, page_content='Ma trận vuông I có tất cả các phần tử trên đường chéo chính bằng 1, các phần tử khác bằng 0.  \nI = [1  0  0]\n[0  1  0]\n[0  0  1]'),
 Document(metadata={'Header 1': 'Kiến thức Ma trận - Đại số Tuyến tính', 'Header 2': '2

In [27]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True}
)


vectorstore = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embedding_model,
    persist_directory=persist_directory,
    collection_name="math_vectors"
)

C:\Users\dongh\AppData\Local\Temp\ipykernel_5688\1942991363.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Test

In [ ]:
# Load model
model = SentenceTransformer(model_name, device=DEVICE)

In [ ]:
# Tạo embedding
texts = [chunk.page_content for chunk in chunks]
embeddings = model.encode(texts, 
                          convert_to_tensor=False,
                          normalize_embeddings=True)

In [10]:
embeddings

array([[ 0.01175354,  0.04333695,  0.01675817, ..., -0.01774279,
        -0.02854558,  0.07406582],
       [ 0.01095208,  0.0504429 , -0.01873817, ..., -0.02134096,
        -0.03130301,  0.03099393]], dtype=float32)